# Day 16 - Maze - part 2

Need to find all the best paths, and mark them.

I will modify in such a way that the shortest paths are stored per node.

Will need a new dictionary with a list of paths.

# Read file as lines

In [1]:
TEST = False

# INPUT_FN = "input_test.txt"
# TEST_SOLUTION = 45

INPUT_FN = "input_test2.txt"
TEST_SOLUTION = 64

if not TEST:
    INPUT_FN = "input.txt"

with open(INPUT_FN) as f:
    lines = [l.rstrip() for l in f]

if lines[-1]=='':
    lines=lines[:,-1]

In [2]:
lines

['#############################################################################################################################################',
 '#.........#.............#.....#.....#.....#.........#...#.............#...#.............#.........#.....#...................#.....#.#......E#',
 '#.#######.#.###########.#.###.###.#.#.#.#.#.#.#.###.###.#.###########.#.###.#####.#######.#.###.#.###.#.#.###########.#.###.###.#.#.#.###.###',
 '#.......................#.#.....#.#.#.#.#.#...#...#...#.#.#.......#...#.......................#.#...............#...........................#',
 '#.#.#.#.#####.#####.#.###.#####.#.#.###.#.###.###.###.#.#.#.#.#.###.###.#.###.#.#.#.#########.#.#####.#######.#.#.#.###.#.###.#.#.###.#####.#',
 '#...#.#.#...#.#...#...#...#...#...#...#.#.........#.....#.#.#.#.#...#...#.#...#.#.....#.....#.#.....#.....#.#.#.#.#.....#.............#.....#',
 '#.###.###.#.###.#.#.#####.#.#.#######.#.#####.#.#######.#.#.#.###.###.###.#.###########.###.#######.#.###.#.#.#.#.#####.#.

In [3]:
MAP=lines

Find start and end

In [4]:
start_pos=None
end_pos = None

for r0,l0 in enumerate(lines):
    if 'S' in l0:
        start_pos = (r0,l0.index('S'))
    if 'E' in l0:
        end_pos = (r0,l0.index('E'))


In [5]:
print(f"{start_pos=}, {end_pos=}")

start_pos=(139, 1), end_pos=(1, 139)


In [6]:
def printd(s):
    if TEST:
        print(s)

Suggested algorithm to solve

BFS whereby visited locations must include the direction of travel

Store the lowest score for each location visited. Have a dictionary to store these.

In [7]:
from collections import deque


best_scores = dict()
visited = []
robot_pos = (*start_pos, 0,+1)
best_scores[robot_pos]= 0

bfs_q = deque()
bfs_q.append(robot_pos)

best_paths_nodes=dict()
best_paths_nodes[robot_pos]=set()

iter=0

while True:
    printd(f"{iter=} , {bfs_q=}")
    if len(bfs_q)==0:
        # no more nodes to visit
        printd("No more nodes to visit")
        break

    curr_pos = bfs_q.popleft()
    curr_score = best_scores[curr_pos]
    cur_best_paths_nodes = best_paths_nodes[curr_pos] # should be a set

    printd(f"{curr_pos=} , {curr_score=}, {cur_best_paths_nodes=}")

    #Check is end position
    if MAP[curr_pos[0]][curr_pos[1]] == 'E':
        printd("End node")
        continue # skip the rest

    #Options are, to move forward or to turn clockwise or anticlockwise
    new_poss = []

    #move forward need to check if there is a wall at new position
    try_pos = (curr_pos[0]+curr_pos[2], curr_pos[1]+curr_pos[3])
    printd(f"move forward to {try_pos=} ?")

    if MAP[try_pos[0]][try_pos[1]] != '#':
        # ok
        printd("can move forward")
        new_poss.append( ( (*try_pos, curr_pos[2], curr_pos[3]) , curr_score+1) )
    
    # rotations
    new_poss.append( ((curr_pos[0], curr_pos[1], -curr_pos[3], curr_pos[2]), curr_score+1000) ) # clockwise
    new_poss.append( ((curr_pos[0], curr_pos[1], curr_pos[3], -curr_pos[2]) , curr_score+1000) ) # anti-clockwise

    # for all these new positions to consider, check if they have been visited
    for npos0,sc0 in new_poss:
        #position visited?

        path_nodes_try = cur_best_paths_nodes.copy()
        path_nodes_try.add(npos0)

        if npos0 in visited:
            # compare scores
            bsc0 = best_scores[npos0]
            b_pn0 = best_paths_nodes[npos0]
            
            if sc0<bsc0:
                printd(f"This path has a better score that previously calculated {sc0=} < {bsc0=}")
                best_scores[npos0] = sc0
                # re-add position to queue to recalculate any scores after this
                bfs_q.append(npos0)

                # best path nodes update
                # This score is better, so replace
                best_paths_nodes[npos0] = path_nodes_try
                
            elif sc0==bsc0:
                # don't need to change scored but does union of the nodes
                best_paths_nodes[npos0] = b_pn0.union(path_nodes_try)

                printd(f"Same score {sc0=} < {bsc0=}, joining path nodes from each")

        else:
            #add this as a new visited place
            printd(f"New visit with {npos0=}")
            visited.append(npos0)
            best_scores[npos0] = sc0
            bfs_q.append(npos0)

            best_paths_nodes[npos0] = path_nodes_try

    iter+=1

This algorithm above does not work well. It is slow because it checks for too many paths. Dijkstra algorithm may be better suited.

There are probably many end positions in the score dictionary, with different directions. I will pick the lowest

In [8]:
best_score_and_paths = None
for k,nodes0 in best_paths_nodes.items():
    if k[0] == end_pos[0] and k[1]==end_pos[1]:
        sc0 = best_scores[k]
        print(f"score {sc0}, best_paths_nodes {k=} , {nodes0=}")
        
        if best_score_and_paths is None:
            best_score_and_paths = (sc0, nodes0)
        else:
            if sc0< best_score_and_paths[0]:
                best_score_and_paths = (sc0,nodes0)
        
print(f"{best_score_and_paths}")

score 88468, best_paths_nodes k=(1, 139, 0, 1) , nodes0={(3, 120, 0, 1), (107, 136, 0, -1), (27, 125, 0, 1), (39, 119, 0, -1), (14, 119, -1, 0), (43, 133, 0, 1), (45, 133, 0, -1), (129, 64, 0, 1), (31, 117, 0, -1), (28, 117, 1, 0), (85, 125, 0, 1), (87, 123, -1, 0), (132, 125, 1, 0), (138, 1, -1, 0), (129, 50, 0, 1), (126, 137, -1, 0), (53, 135, -1, 0), (87, 126, 0, -1), (65, 133, 0, 1), (127, 118, 0, 1), (39, 118, 0, -1), (129, 123, 0, 1), (45, 132, 0, -1), (34, 133, -1, 0), (73, 135, 0, 1), (129, 35, 0, 1), (133, 67, 0, 1), (129, 19, 0, 1), (25, 115, 0, 1), (75, 133, -1, 0), (133, 131, 0, 1), (45, 129, 0, -1), (96, 133, -1, 0), (129, 119, 1, 0), (79, 133, -1, 0), (39, 129, 0, -1), (3, 129, 0, 1), (21, 128, 0, -1), (1, 135, 0, 1), (129, 30, 0, 1), (131, 69, 0, 1), (127, 137, 0, 1), (129, 18, 0, 1), (125, 94, 0, 1), (131, 78, 0, 1), (3, 133, -1, 0), (131, 127, 1, 0), (49, 135, -1, 0), (35, 119, -1, 0), (75, 131, 0, -1), (47, 123, 0, -1), (127, 112, 0, 1), (85, 131, -1, 0), (95, 130, 0,

In [9]:
best_nodes_coords = { n0[:2] for n0 in best_score_and_paths[1]}

In [10]:
score = len(best_nodes_coords) 

In [11]:
print(score)

616


In [12]:
if TEST:
    assert score == TEST_SOLUTION

Debug

In [13]:
best_nodes_coords

{(1, 133),
 (1, 134),
 (1, 135),
 (1, 136),
 (1, 137),
 (1, 138),
 (1, 139),
 (2, 133),
 (2, 137),
 (3, 119),
 (3, 120),
 (3, 121),
 (3, 122),
 (3, 123),
 (3, 124),
 (3, 125),
 (3, 126),
 (3, 127),
 (3, 128),
 (3, 129),
 (3, 130),
 (3, 131),
 (3, 132),
 (3, 133),
 (3, 134),
 (3, 135),
 (3, 136),
 (3, 137),
 (4, 119),
 (5, 119),
 (6, 119),
 (7, 119),
 (8, 119),
 (9, 119),
 (10, 119),
 (11, 119),
 (12, 119),
 (13, 119),
 (14, 119),
 (15, 119),
 (16, 119),
 (17, 119),
 (18, 119),
 (19, 119),
 (20, 119),
 (21, 119),
 (21, 120),
 (21, 121),
 (21, 122),
 (21, 123),
 (21, 124),
 (21, 125),
 (21, 126),
 (21, 127),
 (21, 128),
 (21, 129),
 (21, 130),
 (21, 131),
 (21, 132),
 (21, 133),
 (22, 119),
 (22, 133),
 (23, 119),
 (23, 120),
 (23, 121),
 (23, 122),
 (23, 123),
 (23, 133),
 (24, 123),
 (24, 133),
 (25, 111),
 (25, 112),
 (25, 113),
 (25, 114),
 (25, 115),
 (25, 123),
 (25, 124),
 (25, 125),
 (25, 133),
 (26, 111),
 (26, 115),
 (26, 125),
 (26, 133),
 (27, 111),
 (27, 115),
 (27, 116),
 (

In [14]:
for irow, l0 in enumerate(MAP):
    printline=[]
    for icol, c in enumerate(l0):
        if (irow,icol) in best_nodes_coords:
            printline.append('O')
        else:
            printline.append(c)
    print("".join(printline))

#############################################################################################################################################
#.........#.............#.....#.....#.....#.........#...#.............#...#.............#.........#.....#...................#.....#.#OOOOOOO#
#.#######.#.###########.#.###.###.#.#.#.#.#.#.#.###.###.#.###########.#.###.#####.#######.#.###.#.###.#.#.###########.#.###.###.#.#.#O###O###
#.......................#.#.....#.#.#.#.#.#...#...#...#.#.#.......#...#.......................#.#...............#......OOOOOOOOOOOOOOOOOOO..#
#.#.#.#.#####.#####.#.###.#####.#.#.###.#.###.###.###.#.#.#.#.#.###.###.#.###.#.#.#.#########.#.#####.#######.#.#.#.###O#.###.#.#.###.#####.#
#...#.#.#...#.#...#...#...#...#...#...#.#.........#.....#.#.#.#.#...#...#.#...#.#.....#.....#.#.....#.....#.#.#.#.#....O#.............#.....#
#.###.###.#.###.#.#.#####.#.#.#######.#.#####.#.#######.#.#.#.###.###.###.#.###########.###.#######.#.###.#.#.#.#.#####O#.#######.#.###.#####
#.#.#.